# 使用するデータ
- game_round: 現在の局数(0-7)
- my_seat: 東1局の自分の座席(0-3)
- my_point: 自分の点数
- left_point: 下家の点数
- opposite_point: 対面の点数
- right_point: 上家の点数
- current_result: 現在の順位(0-3)
- game_result : 最終順位(0-3)


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import preprocessing
import pickle

np.set_printoptions(precision=3, suppress=True)

# 予測実行クラス

In [5]:
class Ranking:

    def __init__(self, models=None):
        """
        コンストラクタ
        """
        self.models = models
        self.coeff = 1000

    def train(self):
        df = pd.read_csv('../data/201901_10000game.csv')
        # display('前処理前のデータ',df)

        # 前処理
        df['my_point'] = df['my_point'] / self.coeff
        df['right_point'] = df['right_point'] / self.coeff
        df['opposite_point'] = df['opposite_point'] / self.coeff
        df['left_point'] = df['left_point'] / self.coeff

        df['game_result'] = df['game_result'] - 1
        df = df.astype({'game_result': int, 'my_seat':int})
        df = df.astype({'game_result': str, 'my_seat':str})

        df = pd.get_dummies(df, columns=['my_seat'])

        df = df[['game_round','my_seat_0', 'my_seat_1','my_seat_2', 'my_seat_3',
        'my_point', 'right_point', 'opposite_point', 'left_point', 'game_result']]

        # display('前処理後のデータ',df)

        # 学習
        models = []

        for i in range(8):
            models.append(GradientBoostingClassifier(tol=1e-6,random_state=123, verbose=0))
            model = models[-1]

            x = df[df.game_round == i].iloc[:, 1:9]
            y = df[df.game_round == i].iloc[:, 9]


            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=123)
            
            model.fit(x_train, y_train)

            print('{} 正解率(train) :'.format(i), model.score(x_train, y_train))
            print('{} 正解率(test)  :'.format(i), model.score(x_test, y_test))
        
        with open('gbc_model.bin', mode='wb') as f:
            pickle.dump(models, f)
            
        self.models = models
        print('学習完了')
    
    def predict(self, game_num, my_seat, score):
        """
        予測の実行
        """
        # 入力値の作成
        df = pd.DataFrame({'my_point': score[0]/self.coeff,
                            'right_point': score[1]/self.coeff,
                            'opposite_point': score[2]/self.coeff,
                            'left_point': score[3]/self.coeff},
                            index=['0'])
        df['my_seat_0'] = 1 if my_seat == 0 else 0
        df['my_seat_1'] = 1 if my_seat == 1 else 0
        df['my_seat_2'] = 1 if my_seat == 2 else 0
        df['my_seat_3'] = 1 if my_seat == 3 else 0

        df = df[['my_seat_0', 'my_seat_1','my_seat_2', 'my_seat_3',
        'my_point', 'right_point', 'opposite_point', 'left_point']]

        feature = df.values
        # 予測
        proba = self.models[game_num].predict_proba(feature)[0]
        return proba

# テスト実行

# トレーニング

In [6]:
r = Ranking()
r.train()





0 正解率(train) : 0.30361441216049734
0 正解率(test)  : 0.2829609414975269
1 正解率(train) : 0.37393088635753463
1 正解率(test)  : 0.36175353814290645
2 正解率(train) : 0.41706307769656187
2 正解率(test)  : 0.4110685694396102
3 正解率(train) : 0.46677588204738885
3 正解率(test)  : 0.4564904710535779
4 正解率(train) : 0.5113540617743162
4 正解率(test)  : 0.49287960051784724
5 正解率(train) : 0.5631978367062107
5 正解率(test)  : 0.559850863422292
6 正解率(train) : 0.6323749453281462
6 正解率(test)  : 0.6248187279883985
7 正解率(train) : 0.7288655402527332
7 正解率(test)  : 0.7137745974955277
学習完了


# 読み込み

In [17]:
model = pickle.load(open('lr_model.bin', 'rb'))
r = Ranking(model)

# 自分トップ目

In [18]:

for game_round in range(7):
    f = np.array([350, 300, 200, 150])
    result = r.predict(game_round, 0, f)
    print(result)



[0.464 0.266 0.177 0.094]
[0.439 0.297 0.183 0.081]
[0.457 0.315 0.169 0.059]
[0.479 0.328 0.151 0.042]
[0.499 0.329 0.14  0.032]
[0.465 0.376 0.139 0.02 ]
[0.498 0.39  0.102 0.009]


# 自分ラス目

In [19]:


for game_round in range(7):
    f = np.array([100, 250, 250, 400])
    result = r.predict(game_round, 0, f)
    print(result)

[0.048 0.127 0.247 0.578]
[0.036 0.118 0.254 0.593]
[0.028 0.098 0.263 0.611]
[0.019 0.09  0.266 0.624]
[0.013 0.078 0.262 0.647]
[0.005 0.062 0.267 0.666]
[0.002 0.041 0.26  0.697]


# 2〜3着目

In [20]:


for game_round in range(7):
    f = np.array([250, 250, 400, 100])
    result = r.predict(game_round, 0, f)
    print(result)

[0.252 0.25  0.254 0.244]
[0.222 0.241 0.29  0.247]
[0.222 0.275 0.285 0.217]
[0.218 0.285 0.297 0.199]
[0.207 0.306 0.301 0.186]
[0.15  0.332 0.355 0.163]
[0.129 0.378 0.365 0.128]


# 飛び寸前プレイヤーあり

In [21]:
for game_round in range(7):
    f = np.array([330, 330, 330, 10])
    result = r.predict(game_round, 3, f)
    print(result)

[0.462 0.241 0.193 0.104]
[0.481 0.234 0.192 0.093]
[0.474 0.283 0.172 0.071]
[0.511 0.27  0.162 0.057]
[0.484 0.306 0.164 0.046]
[0.51  0.317 0.146 0.028]
[0.533 0.34  0.114 0.013]


# 横並び

In [22]:
for game_round in range(7):
    f = np.array([240, 250, 250, 260])
    result = r.predict(game_round, 0, f)
    print(result)

[0.221 0.248 0.259 0.272]
[0.194 0.263 0.281 0.262]
[0.193 0.271 0.294 0.242]
[0.184 0.291 0.305 0.221]
[0.173 0.3   0.317 0.21 ]
[0.125 0.329 0.362 0.185]
[0.107 0.359 0.382 0.153]


,game,0,1,2,3,game_result
0,0,0.00,0.00,0.00,0.0,14
1,0,0.58,0.58,1.16,0.0,14
2,1,0.58,-0.22,1.96,0.0,14
3,1,0.38,-0.22,1.66,0.0,14
4,2,1.72,0.92,2.80,0.0,14
...,...,...,...,...,...,...
151520,2,-0.75,1.25,1.00,0.0,6
151521,3,-2.11,2.51,1.00,0.0,6
151522,4,-1.24,3.38,2.74,0.0,6
151523,5,-1.24,2.58,3.54,0.0,6


In [12]:
df = pd.read_csv('../data/201901_1000game.csv')
df = df[df.game_round == 1]
df
# df = df[['my_point', 'right_point', 'opposite_point', 'left_point', 'game_result']]

,current_result,game_result,game_round,left_point,my_point,my_seat,opposite_point,right_point
8,1.0,1.0,1.0,250.0,380.0,0.0,160.0,210.0
9,3.0,3.0,1.0,380.0,210.0,1.0,250.0,160.0
10,4.0,4.0,1.0,210.0,160.0,2.0,380.0,250.0
11,2.0,2.0,1.0,160.0,250.0,3.0,210.0,380.0
40,4.0,1.0,1.0,250.0,173.0,0.0,327.0,250.0
...,...,...,...,...,...,...,...,...
42883,4.0,2.0,1.0,222.0,222.0,3.0,325.0,231.0
42884,2.0,4.0,1.0,180.0,231.0,0.0,222.0,367.0
42885,1.0,3.0,1.0,231.0,367.0,1.0,180.0,222.0
42886,3.0,1.0,1.0,367.0,222.0,2.0,231.0,180.0


In [13]:
df = pd.read_csv('../data/201901_1000game.csv')
df = df[df.game_round == 7]
simple = (df.current_result == df.game_result).sum()
print(simple/len(df))

0.6987951807228916


In [3]:
df = pd.read_csv('../data/201901_1000game.csv')
df = df[df.game_round == 7]
df = df.head(100)
df['my_point'].plot.bar()

<AxesSubplot:>

Error in callback <function flush_figures at 0x7f0acbb1d0d0> (for post_execute):


KeyboardInterrupt: 